## GraphLab
Un modulo/paquete/framework sobre python que integra muy bien con apache spark, haddop, pig, etc y que corre con muchisimos wrappers para problemas de ML. 

Probamos a ver como rinde su clasificacion

In [1]:
## seteamos los parametros del notebook
%autosave 180

import pandas as pd; 
import numpy as np; 
import os;
import random;
import time
#%matplotlib inline
import sys

# Import/Export data from Pandas
import graphlab as gl

np.random.seed(2015)
rootdir=os.getcwd()
rootdir+= "/datasets"
os.chdir(rootdir)

/home/juan/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)
/home/juan/mobility-study/scikit-learn/sklearn/externals/joblib/_multiprocessing_helpers.py:29: UserWarning: [Errno 38] Function not implemented.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


Autosaving every 180 seconds


In [2]:
start_time = time.time()

file_sf = "sf_output.txt.gz"
file_sl = "sl_output.txt.gz"
file_gt = "ground_truth.csv.gz"

sf_sframe = pd.read_csv(file_sf,sep = ",",
                   header = 0,
                     index_col =None)
sl_sframe = pd.read_csv(file_sl,sep = "|",
                   header = 0,
                     index_col = None)
sl_sframe.columns = ["USER" if ("LineKeyOrigin" in col) else col for col in sl_sframe.columns  ]


gt_sframe = pd.read_csv(file_gt,sep = "|",
                   header = 0,
                     index_col = None)

#agrego la etiqueta "_gt" a las columnas del ground_truth
gt_sframe.columns = [col+"_gt" if not("USER" in col) else col for col in gt_sframe.columns  ]

all_time =   time.time() - start_time

print('This cell took %s seconds to run' % all_time)

This cell took 58.2009849548 seconds to run


In [3]:
# para poder usar el estimador MultionmialNB, paso todos los valores negativos en las antenna_ids de sf_table a 0
for col in [col for col in sf_sframe.columns.values if "_ID" in col]:
    sf_sframe[col] = sf_sframe[col].replace(-1,0)

In [4]:
X_sframe = pd.merge(gt_sframe,sf_sframe,how = 'inner', on = 'USER' ) 
X_sframe = pd.merge(sl_sframe,X_sframe,how = 'inner', on = 'USER' )
X_sframe.shape
X_sframe['MOBILITY_DIAMETER'] = X_sframe['MOBILITY_DIAMETER'].astype(int)
#tal vez convenga arrancar sobre una pequenya fraccion del dataset
#X_sframe = X_sframe.sample(frac=0.2)

(2000285, 42)

In [5]:
X_sframe.columns

Index([u'USER', u'CallsWeekDaylight', u'CallsWeekDaylight_EPI',
       u'CallsWeekNight', u'CallsWeekNight_EPI', u'CallsWeekend',
       u'CallsWeekend_EPI', u'TimeWeekDaylight', u'TimeWeekDaylight_EPI',
       u'TimeWeekNight', u'TimeWeekNight_EPI', u'TimeWeekend',
       u'TimeWeekend_EPI', u'TOTAL_USERS', u'EPI_USERS', u'EXP_USERS',
       u'ANTENNA_ID_gt', u'EPIDEMIC_gt', u'EXPOSED_gt', u'EXPOSED',
       u'ANTENNA_ID_0', u'COUNT_0', u'ANTENNA_ID_1', u'COUNT_1',
       u'ANTENNA_ID_2', u'COUNT_2', u'ANTENNA_ID_3', u'COUNT_3',
       u'ANTENNA_ID_4', u'COUNT_4', u'ANTENNA_ID_5', u'COUNT_5',
       u'ANTENNA_ID_6', u'COUNT_6', u'ANTENNA_ID_7', u'COUNT_7',
       u'ANTENNA_ID_8', u'COUNT_8', u'ANTENNA_ID_9', u'COUNT_9', u'EPIDEMIC',
       u'MOBILITY_DIAMETER'],
      dtype='object')

In [8]:
start_time = time.time()


#y_sframe = gl.SFrame(X_sframe[ ["USER"] +[col for col in X_sframe.columns if ("_gt" in col)] ])

X_sframe = gl.SFrame(X_sframe[["EPIDEMIC_gt"] +[col for col in X_sframe.columns if not ("_gt" in col)]])


all_time =   time.time() - start_time

print('This cell took %s seconds to run' % all_time)

2016-04-16 23:59:45,679 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460851184.log


This non-commercial license of GraphLab Create is assigned to svega@dm.uba.ar and will expire on April 16, 2017. For commercial licensing options, visit https://dato.com/buy/.
This cell took 6.80936098099 seconds to run


In [9]:
X_sframe.head()

EPIDEMIC_gt,USER,CallsWeekDaylight,CallsWeekDaylight_EPI,CallsWeekNight,CallsWeekNight_EPI
0,93C92C902199D9A758288F7E50700E22 ...,1,0,0,0
0,9F2F09F4AB0ECF750966EF6325CABB50 ...,18,8,0,0
0,2DE89F4300C150557A28C7C2D7A886CE ...,2,0,1,0
1,7CBEA0D81B3A566E4DC2EC9C96C61723 ...,0,0,0,0
0,642A6766B581EC5BD38B76B557952D2A ...,30,0,1,0
0,74E46DDC2EF298FC924B1AB25F232208 ...,29,0,5,0
1,D9125F84C61CB6EAA2100C14C35235E6 ...,0,0,0,0
0,F9E65653825B3FA7F1EE6E7873411913 ...,3,3,2,2
0,BC1B42970C28821DFB6F1EE31C858CC0 ...,3,0,2,0
1,2A427CDA7FACA280EBA70FCB38BFD9BD ...,0,0,0,0


# Feature engineering
Creamos relaciones entre las columnas, desguasado de algunas categorias tal vez, agrupamos en tachos..

In [10]:
X_sframe.shape

(400057, 40)

In [33]:
X_sframe['MOBILITY_DIAMETER'] = X_sframe['MOBILITY_DIAMETER'].astype(int) 

In [34]:
# partimos en train y test
X_train, X_test = X_sframe.random_split(.65, seed=2016)

#further splits into validation and test
#X_val, X_test = X_test.random_split(.65, seed=2016)

In [35]:
from graphlab.toolkits.feature_engineering import *

quadratic = gl.feature_engineering.create(X_sframe, QuadraticFeatures(excluded_features = ['USER','EPIDEMIC_gt']))

fit_quadratic = quadratic.fit(X_train)
quadratic.save('quadratic_interaction')

quadratic_train = fit_quadratic.transform(X_train)
quadratic_test = fit_quadratic.transform(X_test)


In [36]:
countfeat = gl.feature_engineering.create(X_test, 
               CountFeaturizer(target='EPIDEMIC_gt', excluded_features = ['USER','EPIDEMIC_gt']))

# Transform the train set. This is the dataset I will train my classifier on
transformed_X_train = countfeat.transform(quadratic_train)
transformed_X_test = countfeat.transform(quadratic_test)

# Save the transformer.
countfeat.save('count_features')

## Random Forest Cross Validation

In [40]:
start_time = time.time()

rforest = gl.classifier.random_forest_classifier.create(X_train, target='EPIDEMIC_gt',
                             class_weights = 'auto', num_trees =50, metric = "auc",
            features = [col for col in X_train.column_names() if not ("USER" in col or 'EPIDEMIC_gt' in col)] )

rforest2 = gl.classifier.random_forest_classifier.create(transformed_X_train, target='EPIDEMIC_gt',
                              class_weights = 'auto',num_trees =50, metric = "auc",
            features = [col for col in transformed_X_train.column_names() if not ("USER" in col or 'EPIDEMIC_gt' in col)] )

predictions = rforest.classify(X_test)

predictions2 = rforest2.classify(transformed_X_test)


all_time =   time.time() - start_time

finished_models = True
print('This cell took %s seconds to run' % all_time)

Random forest classifier:

--------------------------------------------------------

Number of examples          : 246746

Number of classes           : 2

Number of feature columns   : 35

...

| 48        | 545.817217   | 0.611619     | 0.600326       |

| 49        | 557.185846   | 0.611568     | 0.600389       |

| 50        | 568.941254   | 0.611811     | 0.600572       |

+-----------+--------------+--------------+----------------+


This cell took 931.042095184 seconds to run


In [42]:
rforest.evaluate(X_test)


{'accuracy': 0.5242825056151734,
 'auc': 0.536171634693168,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        | 27005 |
 |      0       |        1        | 41545 |
 |      1       |        0        | 25172 |
 |      0       |        0        | 46523 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.44737299858358115,
 'log_loss': 0.690711302345696,
 'precision': 0.3939460247994165,
 'recall': 0.5175652107250321,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+-------+
 | threshold | fpr | tpr |   p   |   n   |
 +-----------+-----+-----+-------+-------+
 |    0.0    | 1.0 | 1.0 | 52177 | 88068 |
 |   1e-05   | 1.0 | 1.0 | 52177 | 88068 |
 |   2e-05

In [43]:
rforest2.evaluate(transformed_X_test)


{'accuracy': 0.5221790438161789,
 'auc': 0.6321620401848039,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        | 42058 |
 |      1       |        0        | 10119 |
 |      0       |        0        | 31175 |
 |      0       |        1        | 56893 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.5565877931290032,
 'log_loss': 0.6713159698430494,
 'precision': 0.42503865549615466,
 'recall': 0.8060639745481726,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+-------+
 | threshold | fpr | tpr |   p   |   n   |
 +-----------+-----+-----+-------+-------+
 |    0.0    | 1.0 | 1.0 | 52177 | 88068 |
 |   1e-05   | 1.0 | 1.0 | 52177 | 88068 |
 |   2e-

# Gradient Boosting Cross Validation

In [ ]:
start_time = time.time()

gboost = gl.classifier.boosted_trees_classifier.create(X_train, target='EPIDEMIC_gt',
                             class_weights = 'auto', max_depth =10, metric = "auc", random_seed = 2016,
            features = [col for col in X_train.column_names() if not ("USER" in col or 'EPIDEMIC_gt' in col)] )

gboost2 = gl.classifier.boosted_trees_classifier.create(transformed_X_train, target='EPIDEMIC_gt',
                              class_weights = 'auto', max_depth =10, metric = "auc",random_seed = 2016,
            features = [col for col in transformed_X_train.column_names() if not ("USER" in col or 'EPIDEMIC_gt' in col)] )

predictions = gboost.classify(X_test)

predictions2 = gboost2.classify(transformed_X_test)


all_time =   time.time() - start_time

finished_models = True
print('This cell took %s seconds to run' % all_time)

## Logistic Regression cross validation


In [46]:
start_time = time.time()

lr = gl.classifier.logistic_classifier.create(X_train, target='EPIDEMIC_gt',
                             class_weights = 'auto',  l2_penalty=0.1, l1_penalty=0.01,
                                            max_iterations=60,
            features = [col for col in X_train.column_names() if not ("USER" in col or 'EPIDEMIC_gt' in col)] )

lr2 = gl.classifier.logistic_classifier.create(transformed_X_train, target='EPIDEMIC_gt',
                              class_weights = 'auto',l2_penalty=0.1, l1_penalty=0.01,
                                           max_iterations=60,
            features = [col for col in transformed_X_train.column_names() if not ("USER" in col or 'EPIDEMIC_gt' in col)] )

predictions = lr.classify(X_test)

predictions2 = lr2.classify(transformed_X_test)


all_time =   time.time() - start_time

finished_models = True
print('This cell took %s seconds to run' % all_time)

Logistic regression:

--------------------------------------------------------

Number of examples          : 246785

Number of classes           : 2

Number of feature columns   : 35

...

Tuning step size. First iteration could take longer than subsequent iterations.

| 1         | 2        | 0.000000  | 181.216959   | 0.628688          | 0.631876            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.


This cell took 537.705460072 seconds to run


In [47]:
lr.evaluate(X_test)


{'accuracy': 0.6277799565046882,
 'auc': 0.5238780082192331,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        |   37  |
 |      0       |        1        |   62  |
 |      1       |        0        | 52140 |
 |      0       |        0        | 88006 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.0014155635473257328,
 'log_loss': 0.6896063803087702,
 'precision': 0.37373737373737376,
 'recall': 0.0007091247101213179,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+-------+
 | threshold | fpr | tpr |   p   |   n   |
 +-----------+-----+-----+-------+-------+
 |    0.0    | 1.0 | 1.0 | 52177 | 88068 |
 |   1e-05   | 1.0 | 1.0 | 52177 | 88068 |
 |

In [48]:
lr2.evaluate(transformed_X_test)


{'accuracy': 0.6284074298548968,
 'auc': 0.5341878240433194,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        |  446  |
 |      0       |        1        |  383  |
 |      1       |        0        | 51731 |
 |      0       |        0        | 87685 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.01682828359053692,
 'log_loss': 0.6867272247718981,
 'precision': 0.5379975874547648,
 'recall': 0.008547827586867777,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+-------+
 | threshold | fpr | tpr |   p   |   n   |
 +-----------+-----+-----+-------+-------+
 |    0.0    | 1.0 | 1.0 | 52177 | 88068 |
 |   1e-05   | 1.0 | 1.0 | 52177 | 88068 |
 |   2